In [ ]:
# take 10 videos of cube in bottom right corner and 10 videos of cube in bottom left corner
# for each video, save the first 30 frames as images
# for each image, extract features using the model
# for each feature, train a linear classifier (since we want to see if the features themselves contain info about the corner) to predict the corner of the cube
# assess the accuracy of the classifier on the test set and sort the features by their accuracy

# also idea - analyze success
# after the backbone finished how the net predict where to shift the arm
# can plsy whirh dufferebt values of the arm state given the arm position and see where i teasd to

# think about mechaitsic way to distinguish between data memoriation and generalization

In [ ]:
# plotting all the features for each image

n_features = len(best_features)  # Define the number of features to display
features_per_image = []  # List to hold feature maps for each image

# Collect all specified features for each initial position
for feature_map, img in features_map_of_5_initial_pos:
    feature_maps_for_image = [(feature_map[idx], img) for idx in best_features]
    features_per_image.append(feature_maps_for_image)

# Create a grid with `n_features + 1` rows (1 row for images, and `n_features` for feature maps)
fig, axs = plt.subplots(n_features + 1, 5, figsize=(20, (n_features + 1) * 4))

# Plot images in the first row  

for i, img in enumerate(features_map_of_5_initial_pos):
    axs[0, i].imshow(img[1].squeeze(0).permute(1, 2, 0).numpy())  # Use the image from the tuple
    axs[0, i].set_title(f'Image {i}')
    axs[0, i].axis('off')

# Plot each feature map in subsequent rows
for row_idx, feature_idx in enumerate(best_features, start=1):
    for col_idx, feature_data in enumerate(features_per_image):
        feature_map = feature_data[row_idx - 1][0]  # Access the feature map for this row
        axs[row_idx, col_idx].imshow(feature_map.detach().numpy(), cmap='gray')
        axs[row_idx, col_idx].set_title(f'Feature {feature_idx}')
        axs[row_idx, col_idx].axis('off')

# Add description text in the figure
fig.text(0.5, 0.8, 'ResNet backbone output [15 x 20 x 512] Tensor. Displaying selected features for each initial position (i.e. Tensor[:,:, feature_idx] ).', ha='center')
plt.tight_layout()
plt.show()


In [ ]:
# plotting images as grid


# for vid_index in img_and_features.keys():
#     img, features = img_and_features[vid_index]
#     plt.imshow(img.squeeze().permute(1, 2, 0))
#     plt.show()
#     features = features.squeeze(0).unsqueeze(1).detach()  # Now shape is (512, 15, 20)
#     grid = utils.make_grid(features, nrow=32, normalize=False, padding=2)
#     plt.figure(figsize=(20, 20))
#     plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
#     plt.axis('off')
#     plt.show()
    

# input images collage

imgs = []
for vid_index in img_and_features.keys():
    img, features = img_and_features[vid_index]
    imgs.append(img)

imgs_batch = torch.cat(imgs, dim=0)  # Remove extra channel dimension

# Create the grid
grid = utils.make_grid(imgs_batch, nrow=3, normalize=True, padding=2)

# Plotting the grid
plt.figure(figsize=(20, 20))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.axis('off')
plt.show()


# test_feature = 1
# test_feature_imgs = []
# for vid_index in img_and_features.keys():
#     img, features = img_and_features[vid_index]
#     test_feature_img = features[0,test_feature,:,:].detach()
#     test_feature_imgs.append(test_feature_img)

# # plot the images of the test feature on a grid
# test_feature_imgs = torch.stack(test_feature_imgs).unsqueeze(1)
# grid = utils.make_grid(test_feature_imgs, nrow=3, normalize=False, padding=2)
# plt.figure(figsize=(10, 10))
# plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap='gray')  # use cmap='gray' for grayscale images
# plt.axis('off')
# plt.show()

In [ ]:
# video of grid of all features

import torch
import matplotlib.pyplot as plt
from torchvision import utils
import cv2
import numpy as np

# Function to create and save the video
def create_feature_video(img_and_features, output_file='features_video.mp4'):
    video_frames = []

    # Iterate through each feature (0 to 511)
    for feature_idx in range(512):
        current_imgs = []
        for vid_index in img_and_features.keys():
            img, features = img_and_features[vid_index]
            feature_img = features[0, feature_idx, :, :].detach()
            current_imgs.append(feature_img)

        # Stack images to create a grid for the current feature index
        stacked_imgs = torch.stack(current_imgs).unsqueeze(1)
        grid = utils.make_grid(stacked_imgs, nrow=3, normalize=False, padding=2)
        frame = grid.permute(1, 2, 0).cpu().numpy()

        # Convert to uint8 for OpenCV (normalize to range 0-255)
        frame = (frame - frame.min()) / (frame.max() - frame.min()) * 255
        frame = frame.astype(np.uint8)

        # Ensure the frame is C-contiguous for OpenCV compatibility
        frame = np.ascontiguousarray(frame)

        # Ensure the frame is 3 channels (RGB)
        if frame.ndim == 2 or frame.shape[2] == 1:
            frame = np.stack([frame] * 3, axis=-1)  # Repeat the channel to create RGB

        # Add title (feature number)
        # frame = cv2.putText(frame, f'Feature: {feature_idx}', (10, 50),
        #                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        video_frames.append(frame)

    # Set video writer with size of frame
    height, width, _ = video_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, 10, (width, height))  # 10 frames per second

    # Write each frame to the video
    for frame in video_frames:
        out.write(frame)

    # Release video writer
    out.release()
    print(f"Video saved as {output_file}")

# Call the function to create the video
create_feature_video(img_and_features, output_file='features_video.mp4')

In [ ]:
cam_features = policy.model.backbone(batch["observation.images"][:, cam_index])["feature_map"]
cam_features = cam_features.squeeze(0).detach()  # Now shape is (512, 15, 20)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

observation = {
    "observation.state": torch.randn((6,)).unsqueeze(0).to(device),  # Example robot state with 6 elements
    "observation.images.laptop": torch.rand((3, 480, 640), dtype=torch.float32).unsqueeze(0).to(device),  # Random example image
    "observation.images.phone": torch.rand((3, 480, 640), dtype=torch.float32).unsqueeze(0).to(device),  # Random example image
    # "observation.images.laptop": laptop_1st_frame.to(device),
    # "observation.images.phone": phone_1st_frame.to(device),
}

policy.to(device)
policy.eval()  

with torch.no_grad():
    action = policy.select_action(observation) 

print(action)

batch = policy.normalize_inputs(observation)
batch = dict(batch)  # shallow copy so that adding a key doesn't modify the original
batch["observation.images"] = torch.stack([batch[k] for k in policy.expected_image_keys], dim=-4)


img = batch["observation.images.laptop"]


# show the image
import matplotlib.pyplot as plt
plt.imshow(img.squeeze().permute(1, 2, 0))
plt.show()

for cam_index in range(batch["observation.images"].shape[-4]):
    img = batch["observation.images"][:, cam_index]
    plt.imshow(img.squeeze().permute(1, 2, 0))
    plt.show()
    cam_features = policy.model.backbone(batch["observation.images"][:, cam_index])["feature_map"]
    cam_features = cam_features.squeeze(0).detach()  # Now shape is (512, 15, 20)

    # Create a grid of the feature maps (e.g., 32x16 to fit 512 maps)
    grid = utils.make_grid(cam_features.unsqueeze(1), nrow=32, normalize=False, padding=2)

    # Plot the grid
    plt.figure(figsize=(20, 20))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    plt.axis('off')
    plt.show()

# plot grif of features

# batch["observation.images"].shape == torch.Size([1, 2, 3, 480, 640])

# batch["observation.images"][:, cam_index].shape == torch.Size([1, 3, 480, 640])

# cam_features.shape == torch.Size([1, 512, 15, 20])

# squeeze the batch dimension and permute the dimensions to match the image dimensions

cam_features = cam_features.squeeze(0).permute(1, 2, 0).detach() # shape: (15, 20, 512) # 512 features maps of size 15x20

import torch
import matplotlib.pyplot as plt
import torchvision.utils as utils

# Assuming `cam_features` is your tensor with shape (15, 20, 512)

# Move the feature maps dimension to the first position for plotting



In [ ]:
# load dataset

from lerobot.common.datasets.utils import load_hf_dataset
CODEBASE_VERSION = "v1.6"
split = 'train'
root = None
repo_id = "yanivmel1/new_dataset_cube"
hf_dataset = load_hf_dataset(repo_id, CODEBASE_VERSION, root, split)

In [ ]:
# for full training loop more need to be imported TBD 
from lerobot.common.datasets.utils import cycle

dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=4,
    batch_size=1,
    shuffle=False, # for training set it to True
    sampler=None,
    pin_memory=device.type != "cpu",
    drop_last=False,
)

dl_iter = cycle(dataloader)
for _ in range(1):
    batch = next(dl_iter)
    for key in batch:
        batch[key] = batch[key].to(device, non_blocking=True)

offline_steps = 80000
step = 0
policy.train()
for _ in range(step, offline_steps):
    start_time = time.perf_counter()
    batch = next(dl_iter)
    dataloading_s = time.perf_counter() - start_time
    for key in batch:
        batch[key] = batch[key].to(device, non_blocking=True)
    train_info = update_policy(
        policy,
        batch,
        optimizer,
        cfg.training.grad_clip_norm,
        grad_scaler=grad_scaler,
        lr_scheduler=lr_scheduler,
        use_amp=cfg.use_amp,
    )
    train_info["dataloading_s"] = dataloading_s
    evaluate_and_checkpoint_if_needed(step + 1, is_online=False)
    step += 1


In [1]:
from utils import get_pretrained_model, get_dataset, set_trainable_layers
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import einops

class ACTPolicyWithIntermediate(nn.Module):
    """
    A wrapper around your ACTPolicy that returns the decoder embeddings
    (the final hidden states) *before* the final action_head.
    """
    def __init__(self, original_policy):
        super().__init__()
        self.original_policy = original_policy
        
    def forward(self, batch: dict[str, torch.Tensor]) -> torch.Tensor:
        batch = self.original_policy.normalize_inputs(batch)
        if len(self.original_policy.expected_image_keys) > 0:
            batch = dict(batch)
            batch["observation.images"] = torch.stack(
                [batch[k] for k in self.original_policy.expected_image_keys],
                dim=-4
            )
        batch = self.original_policy.normalize_targets(batch)
        
        encoder_out, encoder_in_pos_embed, decoder_pos_embed = self.original_policy.model.forward_for_extracting_intermediate(batch)
        
        return encoder_out, encoder_in_pos_embed, decoder_pos_embed



In [2]:
policy_repo_id = "yanivmel1/new_dataset_cube_080000"
policy = get_pretrained_model(policy_repo_id)
# policy = set_trainable_layers(policy)

# Wrap it
policy_for_embeddings = ACTPolicyWithIntermediate(policy)
policy_for_embeddings.eval()

# Load your dataset
training_data_repo_id = "yanivmel1/new_dataset_cube"
val_set_repo_id = "yanivmel1/fine_tune_1"
dataset = get_dataset(val_set_repo_id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

# Generate embeddings
all_encoder_out = []
all_encoder_in_pos_embed = []
all_decoder_pos_embed = []
all_labels = []
policy_for_embeddings.to(device)

with torch.no_grad():
    ## add tqdm for progress bar
    i = 0 
    for batch in tqdm.tqdm(data_loader):
        for k, v in batch.items():
            batch[k] = v.to(device)
        # This returns [B, chunk_size, dim_model]
        encoder_out, encoder_in_pos_embed, decoder_pos_embed = policy_for_embeddings(batch)
        # Save them
        all_encoder_out.append(encoder_out.cpu())
        all_encoder_in_pos_embed.append(encoder_in_pos_embed.cpu())
        all_decoder_pos_embed.append(decoder_pos_embed.cpu())
        all_labels.append(batch["action"].cpu())  # or whatever label you need
        i += 1
        if i == 3:
            break

# Done: cat & save them as your new dataset
# check if all elements in all_encoder_in_pos_embed are identical
# x = all_encoder_in_pos_embed[0]
# for i in range(1, len(all_encoder_in_pos_embed)):
#     if not torch.equal(x, all_encoder_in_pos_embed[i]):
#         print("not equal - encoder_in_pos_embed")
# y = all_decoder_pos_embed[0]
# for i in range(1, len(all_decoder_pos_embed)):
#     if not torch.equal(y, all_decoder_pos_embed[i]):
#         print("not equal - decoder")

# all_encoder_out = torch.cat(all_encoder_out, dim=0)  # shape [N, chunk_size, dim_model]
# all_encoder_in_pos_embed = torch.cat(all_encoder_in_pos_embed, dim=0)  # shape [N, chunk_size, dim_model]
# all_decoder_pos_embed = torch.cat(all_decoder_pos_embed, dim=0)  # shape [N, chunk_size, dim_model]
# all_labels = torch.cat(all_labels, dim=0)          # shape [N, chunk_size, action_dim]?
# torch.save({"encoder_out": all_encoder_out, "encoder_in_pos_embed": all_encoder_in_pos_embed, "decoder_pos_embed": all_decoder_pos_embed, "labels": all_labels}, "intermediate_dataset.pt")


Loading weights from local directory


Fetching 46 files:   0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/295 [00:00<?, ?it/s]

all_cam_pos_embeds shape torch.Size([1, 512, 15, 40])
encoder_in_pos_embed list  602
encoder_in_pos_embed - before torch.Size([1, 512])
encoder_in_pos_embed - after torch.Size([602, 1, 512])


  0%|          | 1/295 [21:56<107:31:06, 1316.55s/it]

all_cam_pos_embeds shape torch.Size([1, 512, 15, 40])
encoder_in_pos_embed list  602
encoder_in_pos_embed - before torch.Size([1, 512])
encoder_in_pos_embed - after torch.Size([602, 1, 512])


  1%|          | 2/295 [22:07<44:38:32, 548.51s/it]  

all_cam_pos_embeds shape torch.Size([1, 512, 15, 40])
encoder_in_pos_embed list  602
encoder_in_pos_embed - before torch.Size([1, 512])
encoder_in_pos_embed - after torch.Size([602, 1, 512])


  1%|          | 2/295 [22:18<54:29:09, 669.45s/it]


In [15]:
# len(all_encoder_in_pos_embed) == 3
# (all_encoder_in_pos_embed[2] == all_encoder_in_pos_embed[1]).all()
all_encoder_in_pos_embed[0].shape

torch.Size([602, 1, 512])

In [14]:
# len(all_encoder_out) == 3
all_decoder_pos_embed[0].shape

AttributeError: 'Embedding' object has no attribute 'shape'

In [1]:
import torch
data = torch.load("val_embeddings.pt")

# Print the dimensions of each tensor
for key, value in data.items():
    if isinstance(value, torch.Tensor):
        print(f"{key}: {value.shape}")
    else:
        print(f"{key}: Not a tensor")


encoder_out: torch.Size([602, 9421, 512])
encoder_in_pos_embed: torch.Size([602, 295, 512])
decoder_pos_embed: torch.Size([295, 100, 512])
labels: torch.Size([9421, 100, 6])
action_is_pad: torch.Size([9421, 100])


In [6]:
data["action_is_pad"].

tensor([False,  True])

In [5]:
test_tensor = data["decoder_pos_embed"]
first_matrix = test_tensor[0, :, :]

# Check if all matrices along dim 1 are equal to the first matrix
are_identical = torch.all(test_tensor == first_matrix.unsqueeze(1))

if are_identical:
    print("All matrices are identical.")
else:
    print("Not all matrices are identical.")


Not all matrices are identical.


Loading weights from local directory
normalize_inputs.buffer_observation_images_laptop.mean: requires_grad=False
normalize_inputs.buffer_observation_images_laptop.std: requires_grad=False
normalize_inputs.buffer_observation_images_phone.mean: requires_grad=False
normalize_inputs.buffer_observation_images_phone.std: requires_grad=False
normalize_inputs.buffer_observation_state.mean: requires_grad=False
normalize_inputs.buffer_observation_state.std: requires_grad=False
normalize_targets.buffer_action.mean: requires_grad=False
normalize_targets.buffer_action.std: requires_grad=False
unnormalize_outputs.buffer_action.mean: requires_grad=False
unnormalize_outputs.buffer_action.std: requires_grad=False
model.vae_encoder.layers.0.self_attn.in_proj_weight: requires_grad=False
model.vae_encoder.layers.0.self_attn.in_proj_bias: requires_grad=False
model.vae_encoder.layers.0.self_attn.out_proj.weight: requires_grad=False
model.vae_encoder.layers.0.self_attn.out_proj.bias: requires_grad=False
mode

Fetching 106 files:   0%|          | 0/106 [00:00<?, ?it/s]

Fetching 46 files:   0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/5 [01:19<?, ?it/s]


KeyboardInterrupt: 